In [1]:
import os

In [2]:
%pwd

'/Users/drjodyannjones/Documents/MLEngineering/End-to-End-Cancer-Classification-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/drjodyannjones/Documents/MLEngineering/End-to-End-Cancer-Classification-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            
            create_directories([self.config.artifacts_root])
            
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [16]:
import os
import zipfile
import gdown
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        """
        Fetch data from url
        """
        
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from  {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion =  DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-28 12:54:51,060: INFO: common]: yaml file: config/config.yaml loaded successfully]
[2024-03-28 12:54:51,061: INFO: common]: yaml file: params.yaml loaded successfully]
[2024-03-28 12:54:51,062: INFO: common]: created directory at: artifacts]
[2024-03-28 12:54:51,062: INFO: common]: created directory at: artifacts/data_ingestion]
[2024-03-28 12:54:51,063: INFO: 2415271389]: Downloading data from  https://drive.google.com/file/d/1XzHZBjEzqSm6f7rYnG4fBDpD1rLtOcJF/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1XzHZBjEzqSm6f7rYnG4fBDpD1rLtOcJF
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1XzHZBjEzqSm6f7rYnG4fBDpD1rLtOcJF&confirm=t&uuid=cdad11f4-3554-4f46-bc91-a5994439649b
To: /Users/drjodyannjones/Documents/MLEngineering/End-to-End-Cancer-Classification-Project/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:03<00:00, 12.6MB/s]

[2024-03-28 12:54:56,353: INFO: 2415271389]: Downloaded data from https://drive.google.com/file/d/1XzHZBjEzqSm6f7rYnG4fBDpD1rLtOcJF/view?usp=sharing into file artifacts/data_ingestion/data.zip]
